Tutorial
Import libraries


In [39]:
import torch
from torch.utils.data import Dataset
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import numpy as np
import glob
from sklearn.model_selection import StratifiedShuffleSplit
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR
import argparse
from pedestrian_detection_utils import *

Define arguments, check system

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--learning_rate', type=float, default=1e-4, help='initial learning rate')
parser.add_argument('--batch_size', type=int, default=2,
                    help='batch size,  number of images in each iteration during training')
parser.add_argument('--epochs', type=int, default=100, help='total epochs')
parser.add_argument('--val_split', type=float, default=0.2, help='val split')
parser.add_argument('--test_split', type=float, default=0.2, help='test split')
parser.add_argument('--best_model_path', type=str, help='best model path')
parser.add_argument('--images_path', type=str, help='path to images')
parser.add_argument('--verbose', type=bool, default=True, help='verbose debugging flag')
parser.add_argument('--transfer_learning', type=bool, default=True, help='transfer learning flag')

args = parser.parse_args("")

# Check if GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

if args.verbose:
    print(device)

print(args)

Instantiate arguments, prepare data

In [ ]:
args.images_path="path to images"
# images = glob.glob(args.images_path)